In [1]:
import nltk
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.experimental import enable_iterative_imputer
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.feature_selection import RFE, RFECV, f_regression, chi2
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import GridSearchCV, KFold, RandomizedSearchCV, HalvingGridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.preprocessing import PowerTransformer, QuantileTransformer
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.neighbors import  KNeighborsClassifier
from datetime import datetime as dt
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
from imblearn.combine import SMOTETomek
from collections import Counter
from mlxtend.feature_selection import ColumnSelector
import warnings
# Settings the warnings to be ignored
warnings.filterwarnings('ignore')

In [2]:
# Displaying all columns
pd.set_option('display.max_columns', None)

#Display all data in columns
# pd.set_option("display.max_colwidth", None)

# np.set_printoptions(threshold=None, edgeitems=None, linewidth=None, suppress=None)
# pd.set_option("display.max_rows", None)
# pd.set_option("display.expand_frame_repr", True)
# pd.set_option('display.width', 1000)


In [3]:
#Storing Accuracy

data_accuracy = pd.DataFrame(columns=['Algo', 'Accuracy(KFold)', 'Test', 'Train', 'Recall', 'Precision', 'F1', 'ROC_AUC_SCORE'])
def store(data):
    data_accuracy.loc[len(data_accuracy.index)] = data
    data_accuracy.to_csv('loan_prediction_metric_scores.csv')
    print(data_accuracy)



def get_info_pipeline(pipe_name):
    sub_pipe = pipe_name[:-1]
    transformed_df = pd.DataFrame(
        data=sub_pipe.fit_transform(X_train, y_train),
        columns=sub_pipe.get_feature_names_out(),
    )
    print(transformed_df)

In [4]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [5]:
df1 = pd.read_csv('tmdb_5000_credits.csv')
df2 = pd.read_csv('tmdb_5000_movies.csv')

In [6]:
df1.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [7]:
#Renaming the 'movie_id' col in df1 to 'id' of df2 

In [8]:
df1.rename(columns = {'movie_id':'id'}, inplace = True)

In [9]:
df1.head(1)

,id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [10]:
#joining the credits amd movie dataframes

In [11]:
df2 = pd.merge(df1, df2, on='id')

In [12]:
df2.head(1)

,id,title_x,cast,crew,budget,genres,homepage,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title_y,vote_average,vote_count
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [13]:
#Drpping columns

In [14]:
df2.columns

Index(['id', 'title_x', 'cast', 'crew', 'budget', 'genres', 'homepage',
       'keywords', 'original_language', 'original_title', 'overview',
       'popularity', 'production_companies', 'production_countries',
       'release_date', 'revenue', 'runtime', 'spoken_languages', 'status',
       'tagline', 'title_y', 'vote_average', 'vote_count'],
      dtype='object')

In [15]:
df2.drop(columns=['status', 'runtime'], axis=1, inplace=True)

In [16]:
df2.drop(columns=['title_x'], axis=1, inplace=True)

In [17]:
df2.head(1)

,id,cast,crew,budget,genres,homepage,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,spoken_languages,tagline,title_y,vote_average,vote_count
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Enter the World of Pandora.,Avatar,7.2,11800


In [18]:
df2.drop(columns=['title_y'], axis=1, inplace=True)

In [19]:
df2.shape

(4803, 19)

In [20]:
df2.head(1)

,id,cast,crew,budget,genres,homepage,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,spoken_languages,tagline,vote_average,vote_count
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Enter the World of Pandora.,7.2,11800


In [21]:
df2.drop(columns=['revenue', 'release_date'], axis=1, inplace=True)

In [22]:
df2.drop(columns=['budget', 'homepage', 'popularity', 'spoken_languages', 'production_countries', 'vote_average', 'vote_count'], axis=1, inplace=True)

In [23]:
df2.shape

(4803, 10)

In [24]:
df2.head(1)

,id,cast,crew,genres,keywords,original_language,original_title,overview,production_companies,tagline
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""name"": ""Ingenious Film Partners"", ""id"": 289...",Enter the World of Pandora.


In [25]:
df2.isnull().sum()

id                        0
cast                      0
crew                      0
genres                    0
keywords                  0
original_language         0
original_title            0
overview                  3
production_companies      0
tagline                 844
dtype: int64

In [26]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 0 to 4802
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    4803 non-null   int64 
 1   cast                  4803 non-null   object
 2   crew                  4803 non-null   object
 3   genres                4803 non-null   object
 4   keywords              4803 non-null   object
 5   original_language     4803 non-null   object
 6   original_title        4803 non-null   object
 7   overview              4800 non-null   object
 8   production_companies  4803 non-null   object
 9   tagline               3959 non-null   object
dtypes: int64(1), object(9)
memory usage: 412.8+ KB


In [27]:
df2.drop(columns='production_companies', axis=1, inplace=True)

In [28]:
df2.head(1)

,id,cast,crew,genres,keywords,original_language,original_title,overview,tagline
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.


In [29]:
df2.shape

(4803, 9)

In [30]:
#Dropping null rows

In [31]:
df2.isnull().sum()

id                     0
cast                   0
crew                   0
genres                 0
keywords               0
original_language      0
original_title         0
overview               3
tagline              844
dtype: int64

In [32]:
df2.drop(columns='tagline', axis=1, inplace=True)

In [33]:
df2.dropna(inplace=True)

In [34]:
df2.head(1)

,id,cast,crew,genres,keywords,original_language,original_title,overview
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di..."


In [35]:
df2.isnull().sum()

id                   0
cast                 0
crew                 0
genres               0
keywords             0
original_language    0
original_title       0
overview             0
dtype: int64

In [36]:
#Preprocessing

In [37]:
##Genre

In [38]:
df2.iloc[0, 3]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [39]:
import ast

In [40]:
def fetch_genres(string):
    ls = []
    for i in  ast.literal_eval(string):
        ls.append(i['name'])
    return ls

In [41]:
df2['genres'] = df2['genres'].apply(fetch_genres)

In [42]:
df2.head(1)

,id,cast,crew,genres,keywords,original_language,original_title,overview
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di..."


In [43]:
df2.iloc[3, 3]

['Action', 'Crime', 'Drama', 'Thriller']

In [44]:
##Crew

In [45]:
def fetch_dir_and_prod(string):
    ls = []
    for i in  ast.literal_eval(string):
        if i['job']=='Director':
            ls.append(i['name'])
            break
    return ls

In [46]:
df2.head(1)

,id,cast,crew,genres,keywords,original_language,original_title,overview
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di..."


In [47]:
msg = " "
msg = df2.loc[4].crew

In [48]:
msg

'[{"credit_id": "52fe479ac3a36847f813eaa3", "department": "Writing", "gender": 2, "id": 7, "job": "Screenplay", "name": "Andrew Stanton"}, {"credit_id": "52fe479ac3a36847f813ea65", "department": "Directing", "gender": 2, "id": 7, "job": "Director", "name": "Andrew Stanton"}, {"credit_id": "58c69e819251411d30017673", "department": "Crew", "gender": 2, "id": 7879, "job": "Thanks", "name": "John Lasseter"}, {"credit_id": "52fe479ac3a36847f813eac5", "department": "Production", "gender": 2, "id": 490, "job": "Producer", "name": "Colin Wilson"}, {"credit_id": "58c68f2b9251411d30016cf2", "department": "Production", "gender": 2, "id": 2073, "job": "Casting", "name": "Gail Stevens"}, {"credit_id": "58c68ef19251411d62016ec3", "department": "Sound", "gender": 2, "id": 3996, "job": "Sound Re-Recording Mixer", "name": "Tom Johnson"}, {"credit_id": "52fe479ac3a36847f813ead7", "department": "Editing", "gender": 2, "id": 6958, "job": "Editor", "name": "Eric Zumbrunnen"}, {"credit_id": "58c69a9ec3a3684

In [49]:
import ast
msg = ast.literal_eval(msg)

In [50]:
for i in msg:
    if i['job']=='Director':
        print(i['name'])
        break

Andrew Stanton


In [51]:
df2['crew'] = df2['crew'].apply(fetch_dir_and_prod)

In [52]:
df2.head(1)

,id,cast,crew,genres,keywords,original_language,original_title,overview
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...",[James Cameron],"[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di..."


In [53]:
##keywords

In [54]:
msg = df2.iloc[0, 4]

In [55]:
msg

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [56]:
def fetch_keyowrds(string):
    ls = []
    for i in  ast.literal_eval(string):
        ls.append(i['name'])
            
    return ls

In [57]:
fetch_keyowrds(msg)

['culture clash',
 'future',
 'space war',
 'space colony',
 'society',
 'space travel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alien planet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'love affair',
 'anti war',
 'power relations',
 'mind and soul',
 '3d']

In [58]:
df2['keywords'] = df2['keywords'].apply(fetch_genres)

In [59]:
df2.head(1)

,id,cast,crew,genres,keywords,original_language,original_title,overview
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...",[James Cameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di..."


In [60]:
##cast (first 3)

In [61]:
msg = df2.iloc[0, 1]

In [62]:
msg

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [63]:
def fetch_crew_first_three(string):
    ls = []
    cnt = 3
    for i in  ast.literal_eval(string):
        if cnt==0:
            break
        else:
            ls.append(i['name'])
        cnt=cnt-1
    return ls

In [64]:
msg

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [65]:
fetch_crew_first_three(msg)

['Sam Worthington', 'Zoe Saldana', 'Sigourney Weaver']

In [66]:
df2['cast'] = df2['cast'].apply(fetch_crew_first_three)

In [67]:
df2.head(1)

,id,cast,crew,genres,keywords,original_language,original_title,overview
0,19995,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di..."


In [68]:
df2.rename(columns = {'crew':'director'}, inplace = True)

In [69]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview
0,19995,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di..."


In [70]:
#Preprocessing 'overview' column

#Lowercasing

df2['overview'] = df2['overview'].str.lower()


# Removing HTML tags

import re

def remove_html(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'', text)

df2['overview'] = df2['overview'].apply(remove_html)
# print(df.head())


#Remove @

def remove_at_the_rate(text):

  ls = []
  new = []

  ls = nlp(text)

  for word in ls:
        if word.text != "@":
            new.append(word.text)

  return ' '.join(new)

df2['overview'] = df2['overview'].apply(remove_at_the_rate)



#Removing URL

import re

def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'', text)

df2['overview'] = df2['overview'].apply(remove_url)
# print(df.head())


#Remmove punctuation

import string

punc = string.punctuation

def  remove_punc(text):

    return text.translate(str.maketrans('', '', punc))

df2['overview'] = df2['overview'].apply(remove_punc)





from autocorrect import Speller

check = Speller()

def check_spell(text):

  return check(text)

df2['overview'] = df2['overview'].apply(check_spell)


# Removing stop words


from nltk.corpus import stopwords

stopwords = stopwords.words('english')

def remove_stop_words(text):
    ls = []
    new = []

    ls = nlp(text)

    for word in ls:
        if word.text not in stopwords:

            new.append(word.text)

    return ' '.join(new)

df2['overview'] = df2['overview'].apply(remove_stop_words)


#Removing Contradictions

import contractions

def remove_contradictions(text):

  return " ".join([contractions.fix(word.text) for word in nlp(text)])

df2['overview'] = df2['overview'].apply(remove_contradictions)



def Lemmetization(text):

  return " ".join([word.lemma_ for word in nlp(text)])


# #Stemming

# from nltk.stem.porter import PorterStemmer

# ps = PorterStemmer()
# def stemming(text):

#     return " ".join([ps.stem(word.text) for word in nlp(text)])

df2['overview'] = df2['overview'].apply(Lemmetization)




In [71]:
df2.iloc[0, 7]

'22nd century     paraplegic marine dispatch moon pandora unique mission     become torn follow order protect alien civilization'

In [72]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview
0,19995,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,Avatar,22nd century paraplegic marine dispatch mo...


In [73]:
df2['tags'] = df2.sum(axis=1)

In [74]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview,tags
0,19995,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,Avatar,22nd century paraplegic marine dispatch mo...,19995


In [75]:
#Removing spaces between names in cast and crew column

In [76]:
def remove_spaces(ls):
    for i in range(len(ls)):
        ls[i] = ls[i].replace(' ', '')
    return ls
        # print(ls[i])

In [77]:
ls1 = df2.iloc[0, 1]

In [78]:
ls1

['Sam Worthington', 'Zoe Saldana', 'Sigourney Weaver']

In [79]:
remove_spaces(ls1)

['SamWorthington', 'ZoeSaldana', 'SigourneyWeaver']

In [80]:
df2.drop(columns='tags', axis=1, inplace=True)

In [81]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview
0,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[James Cameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,Avatar,22nd century paraplegic marine dispatch mo...


In [82]:
df2['cast'] = df2['cast'].apply(remove_spaces)

In [83]:
df2['director'] = df2['director'].apply(remove_spaces)

In [84]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview
0,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,Avatar,22nd century paraplegic marine dispatch mo...


In [85]:
df2['combined'] = df2['cast'] + df2['director'] + df2['genres'] + df2['keywords'] 

In [86]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview,combined
0,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,Avatar,22nd century paraplegic marine dispatch mo...,"[SamWorthington, ZoeSaldana, SigourneyWeaver, ..."


In [87]:
msg = df2.iloc[0, 7]

In [88]:
df2.iloc[0, 8]

['SamWorthington',
 'ZoeSaldana',
 'SigourneyWeaver',
 'JamesCameron',
 'Action',
 'Adventure',
 'Fantasy',
 'Science Fiction',
 'culture clash',
 'future',
 'space war',
 'space colony',
 'society',
 'space travel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alien planet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'love affair',
 'anti war',
 'power relations',
 'mind and soul',
 '3d']

In [89]:
def replace_big_spaces(string):
    string = string.replace('  ', '')
    return string

In [90]:
replace_big_spaces(msg)

'22nd century paraplegic marine dispatch moon pandora unique mission become torn follow order protect alien civilization'

In [91]:
df2['overview'] = df2['overview'].apply(replace_big_spaces)

In [92]:
df2.head(5)

,id,cast,director,genres,keywords,original_language,original_title,overview,combined
0,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,Avatar,22nd century paraplegic marine dispatch moon p...,"[SamWorthington, ZoeSaldana, SigourneyWeaver, ..."
1,285,"[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...",en,Pirates of the Caribbean: At World's End,captain barbossa long believe dead come back l...,"[JohnnyDepp, OrlandoBloom, KeiraKnightley, Gor..."
2,206647,"[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes],"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",en,Spectre,cryptic message bond ’s past send trail uncove...,"[DanielCraig, ChristophWaltz, LéaSeydoux, SamM..."
3,49026,"[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan],"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...",en,The Dark Knight Rises,follow death district attorney harvey dent bat...,"[ChristianBale, MichaelCaine, GaryOldman, Chri..."
4,49529,"[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton],"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...",en,John Carter,john carter war weary former military captain ...,"[TaylorKitsch, LynnCollins, SamanthaMorton, An..."


In [93]:
msg = df2.iloc[0, 7]

In [94]:
msg

'22nd century paraplegic marine dispatch moon pandora unique mission become torn follow order protect alien civilization'

In [95]:
def list_to_string(ls):
    st=""
    
    for i in ls:
        st = ' '.join(ls)
    return st

In [96]:
msg = df2.iloc[0, 8]

In [97]:
msg

['SamWorthington',
 'ZoeSaldana',
 'SigourneyWeaver',
 'JamesCameron',
 'Action',
 'Adventure',
 'Fantasy',
 'Science Fiction',
 'culture clash',
 'future',
 'space war',
 'space colony',
 'society',
 'space travel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alien planet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'love affair',
 'anti war',
 'power relations',
 'mind and soul',
 '3d']

In [98]:
list_to_string(msg)

'SamWorthington ZoeSaldana SigourneyWeaver JamesCameron Action Adventure Fantasy Science Fiction culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul 3d'

In [99]:
df2['combined'] = df2['combined'].apply(list_to_string)

In [100]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview,combined
0,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,Avatar,22nd century paraplegic marine dispatch moon p...,SamWorthington ZoeSaldana SigourneyWeaver Jame...


In [101]:
def insert_space_at_beginning(string):
    string = ' ' + string
    return string

In [102]:
df2['overview']=df2['overview'].apply(insert_space_at_beginning)

In [103]:
df2['tags'] = df2['combined'] + df2['overview']

In [104]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview,combined,tags
0,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,Avatar,22nd century paraplegic marine dispatch moon ...,SamWorthington ZoeSaldana SigourneyWeaver Jame...,SamWorthington ZoeSaldana SigourneyWeaver Jame...


In [105]:
df2.iloc[0, 9]

'SamWorthington ZoeSaldana SigourneyWeaver JamesCameron Action Adventure Fantasy Science Fiction culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul 3d 22nd century paraplegic marine dispatch moon pandora unique mission become torn follow order protect alien civilization'

In [106]:
df2['original_language'] = df2['original_language'].apply(insert_space_at_beginning)

In [107]:
df2['tags'] = df2['tags'] + df2['original_language']

In [108]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview,combined,tags
0,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,Avatar,22nd century paraplegic marine dispatch moon ...,SamWorthington ZoeSaldana SigourneyWeaver Jame...,SamWorthington ZoeSaldana SigourneyWeaver Jame...


In [109]:
df2.iloc[0, 9]

'SamWorthington ZoeSaldana SigourneyWeaver JamesCameron Action Adventure Fantasy Science Fiction culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul 3d 22nd century paraplegic marine dispatch moon pandora unique mission become torn follow order protect alien civilization en'

In [110]:
movies = df2[['id', 'original_title', 'tags']]

In [111]:
movies.iloc[0,2]

'SamWorthington ZoeSaldana SigourneyWeaver JamesCameron Action Adventure Fantasy Science Fiction culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul 3d 22nd century paraplegic marine dispatch moon pandora unique mission become torn follow order protect alien civilization en'

In [112]:
movies.iloc[2, 2]

'DanielCraig ChristophWaltz LéaSeydoux SamMendes Action Adventure Crime spy based on novel secret agent sequel mi6 british secret service united kingdom cryptic message bond ’s past send trail uncover sinister organization battle political force keep secret service alive bond feel back layer deceit reveal terrible truth behind spectre en'

In [113]:
msg = movies.iloc[1, 2]

In [114]:
def remove_spaces_str(string):
    string = string.replace(' ', '')
    return string

In [115]:
msg

"JohnnyDepp OrlandoBloom KeiraKnightley GoreVerbinski Adventure Fantasy Action ocean drug abuse exotic island east india trading company love of one's life traitor shipwreck strong woman ship alliance calypso afterlife fighter pirate swashbuckler aftercreditsstinger captain barbossa long believe dead come back life head edge earth turner elizabeth swan nothing quite seem en"

In [116]:
movies.head(1)

,id,original_title,tags
0,19995,Avatar,SamWorthington ZoeSaldana SigourneyWeaver Jame...


In [117]:
movies.iloc[1,2]

"JohnnyDepp OrlandoBloom KeiraKnightley GoreVerbinski Adventure Fantasy Action ocean drug abuse exotic island east india trading company love of one's life traitor shipwreck strong woman ship alliance calypso afterlife fighter pirate swashbuckler aftercreditsstinger captain barbossa long believe dead come back life head edge earth turner elizabeth swan nothing quite seem en"

In [118]:
#Importing the DNN Model

In [119]:
!pip install gensim

In [120]:
import gensim

In [121]:
from gensim.utils import simple_preprocess

In [122]:
msg = movies.iloc[1, 2]

In [123]:
msg

"JohnnyDepp OrlandoBloom KeiraKnightley GoreVerbinski Adventure Fantasy Action ocean drug abuse exotic island east india trading company love of one's life traitor shipwreck strong woman ship alliance calypso afterlife fighter pirate swashbuckler aftercreditsstinger captain barbossa long believe dead come back life head edge earth turner elizabeth swan nothing quite seem en"

In [124]:
msg = remove_punc(msg)

In [125]:
from nltk.tokenize import sent_tokenize, word_tokenize

In [126]:
doc1 = nlp(msg)

In [127]:
doc1

JohnnyDepp OrlandoBloom KeiraKnightley GoreVerbinski Adventure Fantasy Action ocean drug abuse exotic island east india trading company love of ones life traitor shipwreck strong woman ship alliance calypso afterlife fighter pirate swashbuckler aftercreditsstinger captain barbossa long believe dead come back life head edge earth turner elizabeth swan nothing quite seem en

In [128]:
#Removing punctuations from movie's titles

In [129]:
movies['tags'] = movies['tags'].apply(remove_punc)

In [130]:
# movies['tags'] = movies['tags'] + movies['original_title']

In [131]:
msg = movies.iloc[1, 2]

In [132]:
msg

'JohnnyDepp OrlandoBloom KeiraKnightley GoreVerbinski Adventure Fantasy Action ocean drug abuse exotic island east india trading company love of ones life traitor shipwreck strong woman ship alliance calypso afterlife fighter pirate swashbuckler aftercreditsstinger captain barbossa long believe dead come back life head edge earth turner elizabeth swan nothing quite seem en'

In [133]:
doc1 = sent_tokenize(msg)

In [134]:
ls = []
for word in doc1:
    ls.append(simple_preprocess(word))


In [135]:
ls

[['johnnydepp',
  'orlandobloom',
  'keiraknightley',
  'goreverbinski',
  'adventure',
  'fantasy',
  'action',
  'ocean',
  'drug',
  'abuse',
  'exotic',
  'island',
  'east',
  'india',
  'trading',
  'company',
  'love',
  'of',
  'ones',
  'life',
  'traitor',
  'shipwreck',
  'strong',
  'woman',
  'ship',
  'alliance',
  'calypso',
  'afterlife',
  'fighter',
  'pirate',
  'swashbuckler',
  'captain',
  'barbossa',
  'long',
  'believe',
  'dead',
  'come',
  'back',
  'life',
  'head',
  'edge',
  'earth',
  'turner',
  'elizabeth',
  'swan',
  'nothing',
  'quite',
  'seem',
  'en']]

In [136]:

def making_corpus(string):
    corpus = []
    # doc1 = sent_tokenize(string)
    # for word in doc1:
    #     corpus.append(simple_preprocess(word))
    corpus.append(string.split())

    return corpus

In [137]:
msg = movies.iloc[2, 2]

In [138]:
making_corpus(msg)

[['DanielCraig',
  'ChristophWaltz',
  'LéaSeydoux',
  'SamMendes',
  'Action',
  'Adventure',
  'Crime',
  'spy',
  'based',
  'on',
  'novel',
  'secret',
  'agent',
  'sequel',
  'mi6',
  'british',
  'secret',
  'service',
  'united',
  'kingdom',
  'cryptic',
  'message',
  'bond',
  '’s',
  'past',
  'send',
  'trail',
  'uncover',
  'sinister',
  'organization',
  'battle',
  'political',
  'force',
  'keep',
  'secret',
  'service',
  'alive',
  'bond',
  'feel',
  'back',
  'layer',
  'deceit',
  'reveal',
  'terrible',
  'truth',
  'behind',
  'spectre',
  'en']]

In [139]:
movies['tags'] = movies['tags'].apply(making_corpus)

In [140]:
movies['tags'].iloc[0]

[['SamWorthington',
  'ZoeSaldana',
  'SigourneyWeaver',
  'JamesCameron',
  'Action',
  'Adventure',
  'Fantasy',
  'Science',
  'Fiction',
  'culture',
  'clash',
  'future',
  'space',
  'war',
  'space',
  'colony',
  'society',
  'space',
  'travel',
  'futuristic',
  'romance',
  'space',
  'alien',
  'tribe',
  'alien',
  'planet',
  'cgi',
  'marine',
  'soldier',
  'battle',
  'love',
  'affair',
  'anti',
  'war',
  'power',
  'relations',
  'mind',
  'and',
  'soul',
  '3d',
  '22nd',
  'century',
  'paraplegic',
  'marine',
  'dispatch',
  'moon',
  'pandora',
  'unique',
  'mission',
  'become',
  'torn',
  'follow',
  'order',
  'protect',
  'alien',
  'civilization',
  'en']]

In [141]:
corpus = []

In [142]:
def corpus_formation(ls):
    corpus.append(ls)

In [143]:
movies['tags'] = movies['tags'].apply(corpus_formation)

In [144]:
from itertools import chain
flatten_corpus = list(chain.from_iterable(corpus))

In [145]:
for i in range(len(flatten_corpus)):
    if len(flatten_corpus[i]) == 0:
        print(i)

In [146]:
len(flatten_corpus)

4800

In [147]:
#Embeddings

In [148]:
#Google News Word2Vec
import gensim.downloader as api
# model2 = api.load('word2vec-google-news-300')

In [149]:
model = gensim.models.Word2Vec(window=10, min_count=3, workers=8, vector_size=300, alpha=0.025, sg=1, negative=10)

In [150]:
model.build_vocab(flatten_corpus)

In [151]:
model.train(flatten_corpus, total_examples=model.corpus_count, epochs=10)

(2082173, 2367310)

In [152]:
embeddings = model.wv.get_normed_vectors()

In [153]:
#Average Word2Vec for each tags

In [154]:
avg_embeddings = []

In [155]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview,combined,tags
0,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,Avatar,22nd century paraplegic marine dispatch moon ...,SamWorthington ZoeSaldana SigourneyWeaver Jame...,SamWorthington ZoeSaldana SigourneyWeaver Jame...


In [156]:
df2.iloc[0, 9]

'SamWorthington ZoeSaldana SigourneyWeaver JamesCameron Action Adventure Fantasy Science Fiction culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul 3d 22nd century paraplegic marine dispatch moon pandora unique mission become torn follow order protect alien civilization en'

In [157]:
flatten_corpus[0][0]

'SamWorthington'

In [158]:
#Building vocabulary
vocabulary = set(model.wv.index_to_key)

In [159]:
final_embeddings = []
for i in flatten_corpus:   
    avg_embeddings = None
    for j in i:
        
        if j in vocabulary: 
            
            if avg_embeddings is None:
                avg_embeddings = model.wv[j]
            else:
                avg_embeddings = avg_embeddings + model.wv[j]
    if avg_embeddings is not None:
        avg_embeddings = avg_embeddings / len(avg_embeddings)
        final_embeddings.append(avg_embeddings)
# print(len(list(unique_words)))

In [160]:
ls = []
ls.append(model.wv[flatten_corpus[0][0]].tolist())
# from itertools import 
ls = list(chain.from_iterable(ls))

In [161]:
len(ls)

300

In [162]:
len(flatten_corpus[0])

57

In [163]:
len(flatten_corpus)

4800

In [164]:
movies.shape

(4800, 3)

In [165]:
len(final_embeddings)

4800

In [166]:
len(final_embeddings[0])

300

In [167]:
import math

In [168]:
def idx_of_movie_in_dataframe(string):
    idx = movies[movies['original_title'] == string].index.values
    idx = idx.tolist()[0]
    return idx

In [169]:
movies.head(10)

,id,original_title,tags
0,19995,Avatar,None
1,285,Pirates of the Caribbean: At World's End,None
2,206647,Spectre,None
3,49026,The Dark Knight Rises,None
4,49529,John Carter,None
5,559,Spider-Man 3,None
6,38757,Tangled,None
7,99861,Avengers: Age of Ultron,None
8,767,Harry Potter and the Half-Blood Prince,None
9,209112,Batman v Superman: Dawn of Justice,None


In [170]:
idx = idx_of_movie_in_dataframe("Spider-Man")

In [171]:
idx

159

In [172]:
final_embeddings[idx]

array([ 2.27157632e-03, -6.38061902e-03, -2.62224535e-03,  3.53754265e-04,
        1.99535303e-02, -1.85964592e-02,  2.94213812e-03,  2.12038811e-02,
        6.17998792e-03,  1.01175159e-03,  1.12712849e-02, -5.96229360e-03,
       -9.51468106e-03, -1.32300369e-02, -7.34889647e-03, -1.87573712e-02,
        2.70320161e-04,  9.63481702e-03,  1.05701238e-02,  6.08286867e-03,
       -1.95144601e-02, -6.50120899e-04, -8.59838538e-03, -1.11263795e-02,
        9.91087966e-03,  6.84324745e-03, -2.21681111e-02, -1.62198320e-02,
       -1.03924172e-02, -2.45911516e-02,  1.33464020e-02, -1.46304099e-02,
       -6.76345918e-03, -3.86635982e-03,  1.40783498e-02,  1.45611330e-03,
       -9.07718483e-03, -2.54723504e-02, -7.55592191e-04,  1.32391527e-02,
        1.13218417e-02,  5.74464165e-03,  1.26829348e-03, -1.20203486e-02,
       -1.62437689e-02,  2.14107949e-02, -1.38378059e-02,  1.41413370e-02,
        4.53158841e-03,  1.09836971e-02, -1.72863603e-02, -2.73760571e-03,
       -9.69303865e-03,  

In [173]:
#Cosine Similarity

In [174]:
from sklearn.metrics.pairwise import cosine_similarity

In [175]:
movies.shape

(4800, 3)

In [176]:
cosine_similarity(final_embeddings[idx].reshape(1, -1), final_embeddings)

array([[0.7864147 , 0.83210325, 0.74533236, ..., 0.78239256, 0.7075073 ,
        0.7468049 ]], dtype=float32)

In [177]:
len(cosine_similarity(final_embeddings[idx].reshape(1, -1), final_embeddings)[0])

4800

In [178]:
#Getting the top five movies from the inputted movie

In [179]:
movies.head(20)

,id,original_title,tags
0,19995,Avatar,None
1,285,Pirates of the Caribbean: At World's End,None
2,206647,Spectre,None
3,49026,The Dark Knight Rises,None
4,49529,John Carter,None
5,559,Spider-Man 3,None
6,38757,Tangled,None
7,99861,Avengers: Age of Ultron,None
8,767,Harry Potter and the Half-Blood Prince,None
9,209112,Batman v Superman: Dawn of Justice,None


In [211]:
def recommend_movie(index):
    similarity = []
    similarity = cosine_similarity(final_embeddings[index].reshape(1, -1), final_embeddings)[0]
    movie_indexes = list(enumerate(similarity))
    movie_indexes.sort(key = lambda x: x[1], reverse=True)
    movie_indexes = movie_indexes[1:6]
    # print(movie_indexes)
    movie_names = []
    for i in movie_indexes:
        # print(movies.iloc[i[0], 1])
        movie_names.append(movies.iloc[i[0], 1])
        # print(movie)
    return movie_names

In [212]:
idx

159

In [213]:
recommendations = recommend_movie(idx)

In [215]:
recommendations

['Spider-Man 3', 'Spider-Man 2', 'Superman III', 'Mystery Men', 'Sky High']

In [183]:
#Generating requirements.txt

In [184]:
!pip install pipreqs 

In [185]:
# !pipreqs . 

In [186]:
movies.drop(columns='tags', axis=1, inplace=True)

In [187]:
movies.to_csv('movies_refined_dataset.csv')

In [188]:
movies

,id,original_title
0,19995,Avatar
1,285,Pirates of the Caribbean: At World's End
2,206647,Spectre
3,49026,The Dark Knight Rises
4,49529,John Carter
...,...,...
4798,9367,El Mariachi
4799,72766,Newlyweds
4800,231617,"Signed, Sealed, Delivered"
4801,126186,Shanghai Calling


In [189]:
#Saving model

In [190]:
from gensim.models import KeyedVectors   
model.wv.save_word2vec_format('model.bin', binary=True)

In [191]:
#Saving the embeddings in a pickle file

In [192]:
import pickle 
pickle.dump(final_embeddings, open('vectors.pkl', 'wb'))

In [193]:
#Loading the model

In [194]:
# new_model = gensim.models.KeyedVectors.load_word2vec_format('./model.bin', binary=True)

In [195]:
#Testing the saved pickle file


file_name = 'vectors.pkl'
open_file = open(file_name, "rb")
loaded_list = pickle.load(open_file)
open_file.close()

print(loaded_list[0])

[ 1.80218220e-02  3.10174245e-02 -1.31936474e-02  1.91819575e-02
  2.20349785e-02 -2.36254632e-02 -1.41867751e-03  3.98842879e-02
  6.59471704e-03 -1.51579063e-02  7.62276398e-03 -3.97405820e-03
 -6.58559101e-03 -1.66688785e-02 -2.74322405e-02 -3.01815551e-02
  1.61411427e-02 -1.95048265e-02  7.92022794e-03  1.33302566e-02
 -5.22859208e-02  7.92567246e-03 -1.54908532e-02 -2.77196728e-02
  3.30367498e-02 -1.01592869e-03 -3.33010182e-02 -1.98658388e-02
 -1.68824457e-02 -2.38682050e-02  1.29812714e-02 -3.80177866e-03
  1.17164291e-03 -4.42910008e-02  6.45929761e-03 -7.06023909e-03
 -5.24993334e-03 -2.55697798e-02  5.72795374e-03 -7.86720123e-03
  2.60333978e-02 -2.49217544e-03 -1.98224024e-03 -3.10530085e-02
  4.90640290e-03  2.59033069e-02 -3.68575566e-03  2.78189722e-02
  3.00491438e-03 -1.46830324e-04 -2.46944372e-02 -2.10884530e-02
 -1.75428856e-02  3.23679671e-02 -2.50747818e-02  3.83020975e-02
  2.27075536e-02 -3.40704769e-02 -8.34308192e-03  3.93853076e-02
  3.92348797e-04  1.54053